In [15]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

from langchain_openai import OpenAIEmbeddings, OpenAI
from langchain_core.prompts import PromptTemplate

model_name = os.getenv("LLM_MODEL") or "gpt-4o-mini"
model_provider = os.getenv("LLM_PROVIDER") or "openai"

import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate



In [10]:
#### INDEXING ####
loader = WebBaseLoader(
    web_paths=["https://news.naver.com/section/101"],
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("sa_text", "sa_item SECTION_HEADLINE")
        )
    )
)

docs = loader.load()

In [11]:
# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",
    chunk_size=300,
    chunk_overlap=50
)
splits = text_splitter.split_documents(docs)

In [12]:
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(
        model="text-embedding-3-large",
        
    )
)

In [13]:
import re
def clean_whitespace(text):
    """연속된 white space를 하나로 만드는 함수"""
    # 연속된 공백을 하나의 공백으로 줄임
    text = re.sub(r'( )+', ' ', text)
    # 연속된 개행을 하나의 개행으로 줄임
    text = re.sub(r'\n+', '\n', text)
    # 문자열 양쪽 끝의 공백을 제거
    text = text.strip()

    return text

In [14]:
#print(docs)

for doc in docs:
    print(clean_whitespace(doc.page_content))
    print('meta:', doc.metadata)
    print("-"*100)


네이버, 소상공인연합회와 '소상공인 엑스포 in 광주' 개최
네이버(NAVER)가 2023년부터 본격화한 소상공인 지원 노력을 이어간다. 네이버는 클립, 스마트플레이스 등 자사 서비스를 적극 활용해 소상공인 지원을 계속할 방침이다. 네이버는 소상공인연합회와 22일부터 오는 2
머니투데이
17
개의 관련뉴스 더보기
"공공기관 안전경영 운영원칙 법제화…경영평가 안전관리 비중 확대"
최근 한국철도공사(코레일)를 포함해 공공기관에서 안전 사고가 잇따르자 정부가 안전경영을 공공기관 운영 기본 원칙으로 법제화하기로 했다. 공공기관 경영 평가에서 안전관리 비중을 크게 확대하고, 안전관리등급제는 안전성과
아시아경제
22
개의 관련뉴스 더보기
李대통령 첫 방미 경제사절단 15명 확정...삼성, 美 추가 투자 발표 촉각
이재명 대통령과 도널드 트럼프 미국 대통령의 첫 한미 정상회담에 4대 그룹 총수를 비롯 총 15명이 경제사절단으로 동행한다. 삼성전자 등 주요기업들의 '깜짝 투자 발표' 가능성이 제기되는 가운데, 마스가(MASGA·
파이낸셜뉴스
48
개의 관련뉴스 더보기
'외국인 부동산 쇼핑'에 칼뺐다…서울 전역·경기인천 주요지역 '외국인 토허구역' 지정
정부, 원정매수·내국인 역차별 논란에 초강수 규제 실거주 안하면 주택구입 못해…위반땐 강제이행금 ◆ 李정부 부동산 대책 ◆ 정부가 서울 전역과 경기, 인천 주요 지역을 '외국인 토지거래허가구역'으로 지정한다. 안보
매일경제
92
개의 관련뉴스 더보기
[인터뷰] 히스 타버트 서클 사장 “스테이블코인 가장 유용한 형태의 돈… 기존 금융시스템 대안”
한국은 블록체인 잠재력 높은 G20 국가 스테이블코인 USDC, 한국에도 유용해 USDC, CBDC나 각국 법정 화폐와 이상적 조합 현금성 자산 충분해 테라·루나 사태 없을 것 ‘가장 뜨거운 스테이블코인’. 지난 6
조선비즈
43
개의 관련뉴스 더보기
"명품백 대신 살래요" 돌변하더니 '불티'…불황에도 '대박'
니치향수, 명품 화장품, 고가 샴푸 등 스몰 럭셔리 제품 인기가 크게 오르고

In [21]:
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs=dict(
        k=1,
        fetch_k=4
    )
)



In [16]:
template = """당신은 주어진 하나의 질문을 기반으로 여러 검색 쿼리를 생성하는 유용한 조수입니다. 
다음 질문과 관련된 여러 검색 쿼리를 생성하세요: {question}
출력 (4개의 쿼리):"""

prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_rag_fusion
    | ChatOpenAI(model=model_name, temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)




In [19]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60, top_n=2):
    """
    여러 개의 순위가 매겨진 문서 리스트를 받아, RRF(Reciprocal Rank Fusion) 공식을 사용하여 문서의 최종 순위를 계산하는 함수입니다. 
    k는 RRF 공식에서 사용되는 선택적 파라미터이며, top_n은 반환할 우선순위가 높은 문서의 개수입니다.
    """

    fused_scores = {}

    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] = 1 / (rank + 1)

    fused_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    return fused_results[:top_n]

In [22]:
retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion

question = "향후 집갑에 대해서 알려줘"

docs = retrieval_chain_rag_fusion.invoke({"question":question})

/var/folders/jb/wwh7y9vd7zdg6ln544fq5z440000gn/T/ipykernel_49640/3750982584.py:20: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  (loads(doc), score)


In [25]:
len(docs)

2

In [26]:
docs

[(Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='이데일리\n\n33분전\n\n\n\n\n\n\n\n\n래미안 트리니원·아크로 드 서초…하반기 서울 분양 대어\n\n올해 하반기 서울 3352가구 일반분양 반포 신축 대어 ‘래미안 트리니원’도 강남권·역세권 대규모 분양까지 주목 토지거래 허가제, 6·27 대출 규제 등 굵직한 부동산 규제에도 서울 집값 상승이 계속 이뤄지는 가운데\n\n\n매일경제\n\n33분전\n\n\n\n\n\n\n\n\nAI에도 퍼스널컬러가 있나요? 가상모델 ‘갑론을박’ [세모금]'),
  1.0),
 (Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='34분전\n\n\n\n\n\n\n\n\n中 증시 신규 계좌 개설 37%↑…랠리 지속·경기부양 기대감\n\n지원책 발표 지연되면 상승세 조정 가능성도 중국 본토 주식 시장이 10년 만에 최고치를 경신한 가운데, 추가 랠리에 대한 기대감으로 신규 계좌 개설이 급증하고 있다. 22일 홍콩사우스차이나모닝포스트(SCMP)는 올해\n\n\n연합뉴스\n\n35분전\n\n\n\n\n\n\n\n\n콧대 높은 브룩필드…IFC에 이어 청라 물류센터도 매각 무산될 판'),
  1.0)]

In [27]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """다음 맥락을 바탕으로 질문에 답변하세요:

{context}

질문: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(model=model_name, temperature=0)

final_rag_chain = (
    {
        "context": retrieval_chain_rag_fusion,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke(question)

'제공된 기사 내용을 바탕으로 향후 집값 전망에 대해 말씀드리면 다음과 같습니다.\n\n- 서울에서는 올해 하반기에도 강남권과 역세권을 중심으로 대규모 신축 아파트 분양이 예정되어 있습니다. 예를 들어, 반포의 ‘래미안 트리니원’ 등 3,352가구가 일반분양될 예정입니다.\n- 토지거래 허가제, 6·27 대출 규제 등 여러 강력한 부동산 규제에도 불구하고 서울 집값은 계속 상승하는 추세를 보이고 있습니다.\n- 따라서 규제에도 불구하고 서울 주요 지역의 집값은 당분간 상승세가 이어질 가능성이 높다고 볼 수 있습니다.\n\n요약하면, 서울 특히 강남권과 역세권 중심으로 신규 분양 물량이 많고, 규제에도 불구하고 집값 상승세가 지속되고 있어 향후 집값은 상승 기조를 유지할 것으로 예상됩니다. 다만, 부동산 시장은 다양한 변수에 영향을 받으므로 지속적인 시장 동향 관찰이 필요합니다.'